In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/My Drive/ProyectoTesis/AllFeatureData.pkl'


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
allData = pd.read_pickle(file_path)

In [ ]:
featureData = allData.copy()

In [ ]:
features = allData.columns[:-1]
features

In [ ]:
for feature in features:
  featureData[feature] = featureData[feature].apply(lambda x: np.mean(x))

featureData.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
num_data = featureData[features]

In [ ]:
corr = num_data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Create a heatmap
sns.heatmap(corr, annot=True, fmt='.2f', cmap='rocket', square=True, cbar_kws={"shrink": .8})

# Add title

# Show the plot
plt.show()

In [ ]:
for feature in features:
  spoof_data = featureData[featureData["Label"]=="spoof"][feature]
  bonafide_data = featureData[featureData["Label"]=="bonafide"][feature]
     # Create histograms
  plt.hist(spoof_data, alpha=0.5, density=True, label='Spoof')
  plt.hist(bonafide_data, alpha=0.5, density=True, label='Bonafide')

  # Add titles and labels
  plt.title(f'Distribución de {feature}')
  plt.xlabel(feature)
  plt.ylabel('Densidad')
  plt.legend()
  plt.show()


In [ ]:
mean_df = featureData.groupby('Label')
mean_df.mean().T

In [ ]:
import pickle as pkl

In [ ]:
''' path = '/content/drive/My Drive/ProyectoTesis/MeanDataNoFilter.pkl' '''

In [ ]:
''' # Save the object to a file
with open(path, 'wb') as file:  # Open the file in write-binary mode
    pkl.dump(featureData, file) '''

In [ ]:
window_size = 5

prueba = featureData.copy()

In [ ]:
prueba.columns

In [ ]:
prueba.head()

In [ ]:
def moving_average(arr, window_size=5):
    return np.convolve(arr, np.ones(window_size) / window_size, mode='valid')

# Apply to each feature (column in DataFrame)
prueba['zcr'] = prueba['zcr'].rolling(2).mean()


In [ ]:
prueba.head()

# Datos de reconocimiento humano

In [ ]:
import pandas as pd

In [ ]:
path = "/content/drive/My Drive/ProyectoTesis/HumanData.xlsx"

In [ ]:
df = pd.read_excel(path, sheet_name="Raw Data")


In [ ]:
df.drop(df.columns[:20], axis=1, inplace=True)

In [ ]:
edades = {1: '18-25',
          2:'26-35',
          3: '36-45',
          4: '46-55',
          5: '56-65',
          6: '66 o más'
          }

profesiones = {1:'Tecnología/ingeniería',
               2:'Música/sonido',
               3:'Directo con personas',
               4:'Medios Creativos',
               5:'Administación/Negocios',
               6:'Estudiante',
               7:'Otro',
               }

In [ ]:
df.drop(df.columns[18:20], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
arreglar = ['Profesión/Ocupación o interes (Elija todos los que apliquen) - Relacionado con tecnología/ingeniería',
       'Profesión/Ocupación o interes (Elija todos los que apliquen) - Relacionado con música/sonido',
       'Profesión/Ocupación o interes (Elija todos los que apliquen) - Relacionado con trabajo directo con personas ( educación, atención al cliente, salud, psicología, etc.)',
       'Profesión/Ocupación o interes (Elija todos los que apliquen) - Relacionado con medios creativos (diseño, publicidad, cine, etc.)',
       'Profesión/Ocupación o interes (Elija todos los que apliquen) - Administración/Negocios',
       'Profesión/Ocupación o interes (Elija todos los que apliquen) - Estudiante',
       'Profesión/Ocupación o interes (Elija todos los que apliquen) - Otro',]

In [ ]:
def Arreglar(value, count):
  if value == 1:
    return count
  elif type(value) == str:
    return count

In [ ]:
count = 1
for column in arreglar:
  df[column] = df[column].apply(Arreglar, count=count)
  count+=1

In [ ]:
df['Profesion'] = df[arreglar].apply(lambda row: row.tolist(), axis=1)

In [ ]:
df['Profesion']

In [ ]:
classes = {'bonafide': 0, 'spoof':1}
test_y = [0,1,1,0,0,1,1,0,1,0]

In [ ]:
files = [('DF_E_3060166.flac', 'bonafide'),
          ('DF_E_2647211.flac', 'spoof'),
          ('DF_E_3506651.flac', 'spoof'),
          ('DF_E_3554956.flac', 'bonafide'),
          ('DF_E_3578464.flac', 'bonafide'),
          ('DF_E_2387468.flac', 'spoof'),
          ('DF_E_2413090.flac', 'spoof'),
          ('DF_E_4323665.flac', 'bonafide'),
          ('DF_E_4663450.flac', 'spoof'),
          ('DF_E_4465992.flac', 'bonafide')]

In [ ]:
df.columns

In [ ]:
def CreateAgeGraph():
  # Generating the x-axis labels as P1, P2, ..., P10
  x = edades.values()

  # Assuming `questionWise['PercentageCorrect']` contains the y-values
  y = df['Edad'].value_counts().sort_index(ascending=True).values
  print()
  print(df['Edad'].value_counts().index)
  print(x)


  # Plotting the data using Seaborn
  sns.barplot(x=x, y=y, hue=x, palette="viridis", dodge=False, legend=False)
  plt.xlabel('Edades')
  plt.ylabel('Porcentaje de personas')
  plt.show()

CreateAgeGraph()

In [ ]:
examen = df.columns[len(df.columns)-11:-1]

In [ ]:
data = df.copy()

In [ ]:
data.dropna(subset=examen, inplace=True)

In [ ]:
def CreateProfessionGraph():
  from collections import Counter
  # Generating the x-axis labels as P1, P2, ..., P10
  x = profesiones.values()
  arr = []

  y = data['Profesion'].values
  for array in y:
    arr.extend(array)

  arr = np.array(arr)
  arr = arr[~np.isnan(arr)]

  count = Counter(arr)
  count = dict(sorted(count.items(), key=lambda x: x[0], reverse=False))
  y = count.values()

  # Plotting the data using Seaborn
  sns.barplot(x=x, y=y, hue=x, palette="viridis", dodge=False, legend=False)

  plt.xticks(rotation=90)  # Adjust the angle as needed


  plt.xlabel('Profesión')
  plt.ylabel('Porcentaje de personas')
  plt.show()




CreateProfessionGraph()

In [ ]:
data.head()

In [ ]:
for column in examen:
  data[column] = data[column].apply(lambda x: 0 if x == 2 else 1)

In [ ]:
print(data.loc[0, examen].tolist())
print(test_y)

In [ ]:
def AddConfMatrix(value):
  from sklearn.metrics import classification_report, confusion_matrix
  y_pred = value.tolist()
  y_test = [0,1,1,0,0,1,1,0,1,0]

  conf_matrix = confusion_matrix(y_test, y_pred)
  return conf_matrix

In [ ]:
def MetricsDfConverter(classes, metrics):
  cat1, cat2, macro = metrics
  columns = ["Precision", "Recall", "F1-Score"]
  df = pd.DataFrame(metrics, classes, columns)

  for column in columns:
    df[column] = df[column].apply(lambda x: f"{x:.4f}")

  return df

In [ ]:
def MetricsCalculator(confusion_matrix, model=None):
  bonafide, spoof = confusion_matrix

  ps = spoof[1]/(spoof[1]+bonafide[1])
  rs = spoof[1]/sum(spoof)
  f1s = 2*(ps*rs)/(ps+rs)

  s_metrics = [ps,rs,f1s]


  pb = bonafide[0]/(bonafide[0]+spoof[0])

  if np.isnan(pb):
    pb = 0

  rb = bonafide[0]/sum(bonafide)
  f1b = 2*(pb*rb)/(pb+rb)

  if np.isnan(f1b):
    f1b = 0

  b_metrics = [pb,rb,f1b]


  macro_p = (ps+pb)/2
  macro_r = (rs + rb)/2
  macro_f1 = (f1s + f1b)/2

  accuracy = (spoof[1]+bonafide[0])/(sum(bonafide)+sum(spoof))
  macro_metrics = [macro_p, macro_r, macro_f1]


  classes = ["spoof", "bonafide", "Macro-Avg"]
  metrics = [s_metrics, b_metrics, macro_metrics]


  print(f"\n************ {model} ************")
  print(f"Accuracy: {(spoof[1]+bonafide[0])/(sum(bonafide)+sum(spoof)):.4f}\n")
  print(MetricsDfConverter(classes, metrics))
  print("\n**********************************\n")

  return macro_metrics



In [ ]:
def MetricsCalculator2(confusion_matrix, model=None):
  bonafide, spoof = confusion_matrix

  ps = spoof[1]/(spoof[1]+bonafide[1])
  rs = spoof[1]/sum(spoof)
  f1s = 2*(ps*rs)/(ps+rs)

  s_metrics = [ps,rs,f1s]


  pb = bonafide[0]/(bonafide[0]+spoof[0])

  if np.isnan(pb):
    pb = 0

  rb = bonafide[0]/sum(bonafide)
  f1b = 2*(pb*rb)/(pb+rb)

  if np.isnan(f1b):
    f1b = 0

  b_metrics = [pb,rb,f1b]


  macro_p = (ps+pb)/2
  macro_r = (rs + rb)/2
  macro_f1 = (f1s + f1b)/2

  accuracy = (spoof[1]+bonafide[0])/(sum(bonafide)+sum(spoof))
  macro_metrics = [accuracy, macro_p, macro_r, macro_f1]


  classes = ["spoof", "bonafide", "Macro-Avg"]
  metrics = [s_metrics, b_metrics, macro_metrics]


  # print(f"\n************ {model} ************")
  # print(f"Accuracy: {(spoof[1]+bonafide[0])/(sum(bonafide)+sum(spoof)):.4f}\n")
  # print(MetricsDfConverter(classes, metrics))
  # print("\n**********************************\n")

  return macro_metrics


In [ ]:
data['ConfusionMatrix'] = data[examen].apply(AddConfMatrix, axis=1)

In [ ]:
data["ConfusionMatrix"]

### Questionwise

In [ ]:
data['Metrics'] = data['ConfusionMatrix'].apply(MetricsCalculator2)

In [ ]:
data['Accuracy'] = data['Metrics'].apply(lambda x: x[0])
data['Macro-avg-Precision'] = data['Metrics'].apply(lambda x: x[1])
data['Macro-avg-Recall'] = data['Metrics'].apply(lambda x: x[2])
data['Macro-avg-F1-Score'] = data['Metrics'].apply(lambda x: x[3])

In [ ]:
data['Accuracy']
data['Macro-avg-Precision']
data['Macro-avg-Recall']
data['Macro-avg-F1-Score']

In [ ]:
data[data['¿Suele consumir contenido en audio (por ejemplo, podcasts, audiolibros, radio)? - Frecuencia']==5]

In [ ]:
data[data['Accuracy']>=0.7]

In [ ]:
questionWise = data[examen].T.copy()

In [ ]:
questionWise['Bonafide'] = questionWise.apply(lambda row: row.value_counts().tolist()[1], axis=1)
questionWise['Spoof'] = questionWise.apply(lambda row: row.value_counts().tolist()[0], axis=1)

In [ ]:
questionWise.drop([column for column in questionWise.columns if column not in ['Bonafide', 'Spoof']], axis=1, inplace=True)

In [ ]:
questionWise['Answers'] = test_y

In [ ]:
def calcCorrect(value):
  bf, s, ans = value
  tot = bf+s
  if ans == 0:
    return bf/tot
  else:
    return s/tot

In [ ]:
questionWise['PercentageCorrect'] = questionWise.apply(calcCorrect, axis=1)

In [ ]:
questionWise['File'] = [file[0] for file in files]

In [ ]:
questionWise

In [ ]:
def createGraph():
    # Generating the x-axis labels as P1, P2, ..., P10
    x = [f'P{i}' for i in range(1, 11)]

    # Assuming `questionWise['PercentageCorrect']` contains the y-values
    y = questionWise['PercentageCorrect'].values
    print(y*95)
    answers = {0: 'Bonafide', 1: 'Spoof'}
    correct = 0
    for i in range(len(questionWise)):
      ans = questionWise['Answers'].values[i]
      cat = answers[ans]
      correct += questionWise[cat][i]
    print(correct, correct/950)
    avg_value = correct/950


    # Adding a horizontal line for the average
    plt.axhline(avg_value, color='red', linestyle='--', linewidth=1.5, label=f'Promedio: {avg_value:.2f}')

    # Displaying the legend
    plt.legend()


    # Plotting the data using Seaborn
    sns.barplot(x=x, y=y, hue=x, palette="viridis", dodge=False, legend=False)
    plt.xlabel('Preguntas')
    plt.ylabel('Porcentaje de aciertos')
    plt.show()

createGraph()

In [ ]:
len

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix



### Metricas agrupando por edad

In [ ]:
data["Edad"].unique()

In [ ]:
age_confm = []

In [ ]:
for value in data["Edad"].unique():
  answers = data[data["Edad"]==value]
  y_pred = answers[examen].values.flatten()
  y_test = test_y*len(answers)
  conf_matrix = confusion_matrix(y_test, y_pred)
  age_confm.append((conf_matrix, edades[value]))

In [ ]:
metrics_ages = []

In [ ]:
for matrix, age in age_confm:
  metrics = MetricsCalculator(matrix, age)
  metrics_ages.append((metrics, age))

### Metricas agupadas por profesión

In [ ]:
cf_profesions = []

In [ ]:
for value in range(1,7+1):
  answers = data[data['Profesion'].apply(lambda x: value in x)]
  y_pred = answers[examen].values.flatten()
  y_test = test_y*len(answers)
  conf_matrix = confusion_matrix(y_test, y_pred)
  cf_profesions.append((conf_matrix, profesiones[value]))

In [ ]:
cf_profesions

In [ ]:
metrics_profesions = []

In [ ]:
for matrix, profesion in cf_profesions:
  metrics = MetricsCalculator(matrix, profesion)
  metrics_profesions.append((metrics, profesion))

### Metricas agrupadas por genero

In [ ]:
gen_confm = []

In [ ]:
genero = {1: 'Masculino',
          2: 'Femenino'}

In [ ]:
for value in data["Género"].unique()[:-1]:
  answers = data[data["Género"]==value]
  y_pred = answers[examen].values.flatten()
  y_test = test_y*len(answers)
  conf_matrix = confusion_matrix(y_test, y_pred)
  gen_confm.append((conf_matrix, genero[value]))

In [ ]:
gen_confm

In [ ]:
metrics_gender = []

In [ ]:
for matrix, genero in gen_confm:
  metrics = MetricsCalculator(matrix, genero)
  metrics_profesions.append((metrics, genero))

In [ ]:
all_ans = gen_confm[0][0] + gen_confm[1][0]

In [ ]:
all_ans

In [ ]:
MetricsCalculator(all_ans, "Reconocimiento humano")